In [22]:
import numpy as np
import pandas as pd
import math
pd.set_option('display.max_columns', 100)
pd.options.display.float_format = '{:.2f}'.format

# Outline

###  Part A. Affordable  Rentals
 1. Bring in 2018 PUMS data
 2. Assign county_id to each puma
 3. Bring in and clean, reformat county AMIs data
 4. Add variables for cost limits for ELI, VLI, LI, MI, and HI by bedroom count for each county
 5. Add count dummies for each unit based on income and unit type
 

###  Part B. Affordable Owning
 1. Calculate total monthly housing cost by adding monthly first mortgage, second/junior mortgage, and condo fees
 2. Add count dummies for each unit based on income and unit type

### Part C. Weighting Sample to get estimates
 1. Multiply each dummy by the the household weight variable
 2. Calculte standard errors for each estimate
 
### Part D. Aggregating at PUMA Level
 1. Add dummy variables for vacant "for-rent," vacant "for-sale," and recently moved in and create second dataset
 2. Aggregate at PUMA level for each dataset and export

## Part A1. Bring in 2018 PUMS data

In [23]:
variable_types = {"BDSP":"float","RMSP":"str","MV":"float","WGTP":"float","TAXAMT":"float"}
columns = ["PUMA","RT","BDSP","BLD","RNTP","MRGP","SMP","CONP","TEN","VACS","VALP","TAXAMT","GRPIP","GRNTP","HINCP","MV","WGTP"]
pums_df=pd.read_csv("2018_pums.csv", delimiter=",",usecols=columns,dtype=variable_types)
pums_df

,RT,PUMA,WGTP,BDSP,BLD,CONP,MRGP,RNTP,SMP,TEN,VACS,VALP,GRNTP,GRPIP,HINCP,MV,TAXAMT
0,H,3701,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,H,7306,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,H,3755,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,H,7319,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,H,6511,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159680,H,8103,64.00,4.00,2.00,0.00,nan,nan,nan,2.00,nan,900000.00,nan,nan,78000.00,2.00,11500.00
159681,H,5904,121.00,4.00,2.00,0.00,1700.00,nan,nan,1.00,nan,750000.00,nan,nan,105000.00,6.00,3150.00
159682,H,1304,73.00,4.00,2.00,0.00,1200.00,nan,nan,1.00,nan,2211000.00,nan,nan,118900.00,6.00,3450.00
159683,H,1308,78.00,4.00,2.00,0.00,nan,nan,nan,2.00,nan,600000.00,nan,nan,32400.00,4.00,6500.00


In [24]:
pums_df.rename(columns={"RT":"record_type","BDSP":"number_bedrooms",
    "BLD":"units_in_structure",
    "RNTP":"monthly_rent",
    "MRGP":"first_mortgage",
    "SMP":"second_mortgage",
    "CONP":"condo_fee",
    "TEN":"tenure",
    "VACS":"vacancy_status",
    "VALP":"property_value",
    "GRPIP":"gross_rent_pct_of_income",
    "GRNTP":"gross_rent",
    "HINCP":"hh_income",
    "WGTP":"weight",
    "TAXAMT":"prop_tax",
    "MV":"moved_in"}, inplace=True)
pums_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax
0,H,3701,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,H,7306,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,H,3755,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,H,7319,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,H,6511,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159680,H,8103,64.00,4.00,2.00,0.00,nan,nan,nan,2.00,nan,900000.00,nan,nan,78000.00,2.00,11500.00
159681,H,5904,121.00,4.00,2.00,0.00,1700.00,nan,nan,1.00,nan,750000.00,nan,nan,105000.00,6.00,3150.00
159682,H,1304,73.00,4.00,2.00,0.00,1200.00,nan,nan,1.00,nan,2211000.00,nan,nan,118900.00,6.00,3450.00
159683,H,1308,78.00,4.00,2.00,0.00,nan,nan,nan,2.00,nan,600000.00,nan,nan,32400.00,4.00,6500.00


## Part A2. Assign county id to each puma

In [25]:
# load in crosswalk file
crosswalk_df=pd.read_csv("C:/Users/Terner GSR/Box/Cost of Doing Nothing/Climate Impacts/Data/PUMS/PUMA_County_Crosswalk_v2.csv", delimiter=",")
crosswalk_df

,PUMA,county1,county2,county3,county4,county5,county6,county7
0,101,Alameda CA,NaN,NaN,NaN,NaN,NaN,NaN
1,102,Alameda CA,NaN,NaN,NaN,NaN,NaN,NaN
2,103,Alameda CA,NaN,NaN,NaN,NaN,NaN,NaN
3,104,Alameda CA,NaN,NaN,NaN,NaN,NaN,NaN
4,105,Alameda CA,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
260,11103,Ventura CA,NaN,NaN,NaN,NaN,NaN,NaN
261,11104,Ventura CA,NaN,NaN,NaN,NaN,NaN,NaN
262,11105,Ventura CA,NaN,NaN,NaN,NaN,NaN,NaN
263,11106,Ventura CA,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
# add county name column to puma file
puma_county_df=pums_df.merge(crosswalk_df, how='left', left_on = "PUMA",right_on = "PUMA")
puma_county_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7
0,H,3701,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN
1,H,7306,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,San Diego CA,NaN,NaN,NaN,NaN,NaN,NaN
2,H,3755,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN
3,H,7319,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,San Diego CA,NaN,NaN,NaN,NaN,NaN,NaN
4,H,6511,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Riverside CA,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159680,H,8103,64.00,4.00,2.00,0.00,nan,nan,nan,2.00,nan,900000.00,nan,nan,78000.00,2.00,11500.00,San Mateo CA,NaN,NaN,NaN,NaN,NaN,NaN
159681,H,5904,121.00,4.00,2.00,0.00,1700.00,nan,nan,1.00,nan,750000.00,nan,nan,105000.00,6.00,3150.00,Orange CA,NaN,NaN,NaN,NaN,NaN,NaN
159682,H,1304,73.00,4.00,2.00,0.00,1200.00,nan,nan,1.00,nan,2211000.00,nan,nan,118900.00,6.00,3450.00,Contra Costa CA,NaN,NaN,NaN,NaN,NaN,NaN
159683,H,1308,78.00,4.00,2.00,0.00,nan,nan,nan,2.00,nan,600000.00,nan,nan,32400.00,4.00,6500.00,Contra Costa CA,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
# remove " CA" from end of each county name
puma_county_df['County'] = puma_county_df['county1'].str.replace(r' CA', '')
puma_county_df['county2'] = puma_county_df['county2'].str.replace(r' CA', '')
puma_county_df['county3'] = puma_county_df['county3'].str.replace(r' CA', '')
puma_county_df['county4'] = puma_county_df['county4'].str.replace(r' CA', '')
puma_county_df['county5'] = puma_county_df['county5'].str.replace(r' CA', '')
puma_county_df['county6'] = puma_county_df['county6'].str.replace(r' CA', '')
puma_county_df['county7'] = puma_county_df['county7'].str.replace(r' CA', '')
puma_county_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7,County
0,H,3701,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles
1,H,7306,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,San Diego CA,NaN,NaN,NaN,NaN,NaN,NaN,San Diego
2,H,3755,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles
3,H,7319,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,San Diego CA,NaN,NaN,NaN,NaN,NaN,NaN,San Diego
4,H,6511,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Riverside CA,NaN,NaN,NaN,NaN,NaN,NaN,Riverside
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159680,H,8103,64.00,4.00,2.00,0.00,nan,nan,nan,2.00,nan,900000.00,nan,nan,78000.00,2.00,11500.00,San Mateo CA,NaN,NaN,NaN,NaN,NaN,NaN,San Mateo
159681,H,5904,121.00,4.00,2.00,0.00,1700.00,nan,nan,1.00,nan,750000.00,nan,nan,105000.00,6.00,3150.00,Orange CA,NaN,NaN,NaN,NaN,NaN,NaN,Orange
159682,H,1304,73.00,4.00,2.00,0.00,1200.00,nan,nan,1.00,nan,2211000.00,nan,nan,118900.00,6.00,3450.00,Contra Costa CA,NaN,NaN,NaN,NaN,NaN,NaN,Contra Costa
159683,H,1308,78.00,4.00,2.00,0.00,nan,nan,nan,2.00,nan,600000.00,nan,nan,32400.00,4.00,6500.00,Contra Costa CA,NaN,NaN,NaN,NaN,NaN,NaN,Contra Costa


## Part A3. Bring in and clean, reformat county AMIs data

In [28]:
#bring in 2018 county AMIs file
columns=["County","Income_Category","1","2","4","6","8"]
df = pd.read_csv("2018_AMIs.csv", delimiter=",", usecols=columns)
df

,County,Income_Category,1,2,4,6,8
0,Alameda County\n4-Per,Extremely Low,24400,27900,34850,40450,46050
1,Alameda County\n4-Per,Very Low Income,40700,46500,58100,67400,76700
2,Alameda County\n4-Per,Low Income,62750,71700,89600,103950,118300
3,Alameda County\n4-Per,Median Income,73100,83500,104400,121100,137800
4,Alameda County\n4-Per,Moderate Income,87700,100250,125300,145350,165400
...,...,...,...,...,...,...,...
285,Yuba County\n4-Person,Extremely Low,12600,16460,25100,33740,39600
286,Yuba County\n4-Person,Very Low Income,21000,24000,30000,34800,39600
287,Yuba County\n4-Person,Low Income,33600,38400,48000,55700,63400
288,Yuba County\n4-Person,Median Income,42000,48000,60000,69600,79200


In [29]:
#clean county names variable
df['County'] = df['County'].str.replace(r' County\n4-Person', '')
df['County'] = df['County'].str.replace(r' County\n4-Perso', '')
df['County'] = df['County'].str.replace(r' County\n4-Pers', '')
df['County'] = df['County'].str.replace(r' County\n4-Per', '')
df['County'] = df['County'].str.replace(r' County\n4-Pe', '')
df['County'] = df['County'].str.replace(r' County\n4-P', '')
df['County'] = df['County'].str.replace(r' County\n4-', '')
df['County'] = df['County'].str.replace(r' County\n4', '')
df['County'] = df['County'].str.replace(r' County\n', '')
df['County'] = df['County'].str.replace(r' County\ ' , '')
df['County'] = df['County'].str.replace(r' County', '')
df['County'] = df['County'].str.replace(r' Count', '')
df['County'] = df['County'].str.replace(r' Coun', '')
df

,County,Income_Category,1,2,4,6,8
0,Alameda,Extremely Low,24400,27900,34850,40450,46050
1,Alameda,Very Low Income,40700,46500,58100,67400,76700
2,Alameda,Low Income,62750,71700,89600,103950,118300
3,Alameda,Median Income,73100,83500,104400,121100,137800
4,Alameda,Moderate Income,87700,100250,125300,145350,165400
...,...,...,...,...,...,...,...
285,Yuba,Extremely Low,12600,16460,25100,33740,39600
286,Yuba,Very Low Income,21000,24000,30000,34800,39600
287,Yuba,Low Income,33600,38400,48000,55700,63400
288,Yuba,Median Income,42000,48000,60000,69600,79200


In [30]:
#create df for each income level and calculate max rents, rename variables
eli_df = df[df.Income_Category =="Extremely Low"].copy()
eli_df["1"]=eli_df["1"]*0.025
eli_df["2"]=eli_df["2"]*0.025
eli_df["4"]=eli_df["4"]*0.025
eli_df["6"]=eli_df["6"]*0.025
eli_df["8"]=eli_df["8"]*0.025
eli_df.rename(columns={"1":"ELI_studio","2":"ELI_1_br","4":"ELI_2_br","6":"ELI_3_br","8":"ELI_4_br"}, inplace=True)
eli_df.drop('Income_Category', axis=1, inplace=True)
eli_df

,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br
0,Alameda,610.00,697.50,871.25,1011.25,1151.25
5,Alpine,453.75,518.75,647.50,843.50,1059.50
10,Amador,387.50,442.50,627.50,843.50,1059.50
15,Butte,330.00,411.50,627.50,843.50,1033.00
20,Calaveras,380.00,435.00,627.50,843.50,1059.50
25,Colusa,315.00,411.50,627.50,843.50,988.75
30,Contra Costa,610.00,697.50,871.25,1011.25,1151.25
35,Del Norte,315.00,411.50,627.50,843.50,988.75
40,El Dorado,421.25,481.25,627.50,843.50,1059.50
45,Fresno,315.00,411.50,627.50,843.50,988.75


In [31]:
vli_df = df[df.Income_Category =="Very Low Income"].copy()
vli_df["1"]=vli_df["1"]*0.025
vli_df["2"]=vli_df["2"]*0.025
vli_df["4"]=vli_df["4"]*0.025
vli_df["6"]=vli_df["6"]*0.025
vli_df["8"]=vli_df["8"]*0.025
vli_df.rename(columns={"1":"VLI_studio","2":"VLI_1_br","4":"VLI_2_br","6":"VLI_3_br","8":"VLI_4_br"}, inplace=True)
vli_df.drop('Income_Category', axis=1, inplace=True)
vli_df

,County,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br
1,Alameda,1017.50,1162.50,1452.50,1685.00,1917.50
6,Alpine,756.25,865.00,1080.00,1253.75,1426.25
11,Amador,645.00,736.25,920.00,1067.50,1215.00
16,Butte,548.75,626.25,782.50,908.75,1033.75
21,Calaveras,633.75,723.75,903.75,1048.75,1193.75
26,Colusa,525.00,600.00,748.75,868.75,988.75
31,Contra Costa,1017.50,1162.50,1452.50,1685.00,1917.50
36,Del Norte,525.00,600.00,748.75,868.75,988.75
41,El Dorado,701.25,801.25,1001.25,1162.50,1322.50
46,Fresno,525.00,600.00,748.75,868.75,988.75


In [32]:
li_df = df[df.Income_Category =="Low Income"].copy()
li_df["1"]=li_df["1"]*0.025
li_df["2"]=li_df["2"]*0.025
li_df["4"]=li_df["4"]*0.025
li_df["6"]=li_df["6"]*0.025
li_df["8"]=li_df["8"]*0.025
li_df.rename(columns={"1":"LI_studio","2":"LI_1_br","4":"LI_2_br","6":"LI_3_br","8":"LI_4_br"}, inplace=True)
li_df.drop('Income_Category', axis=1, inplace=True)
li_df

,County,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br
2,Alameda,1568.75,1792.50,2240.00,2598.75,2957.50
7,Alpine,1152.50,1316.25,1645.00,1908.75,2172.50
12,Amador,1031.25,1178.75,1472.50,1708.75,1943.75
17,Butte,877.50,1002.50,1252.50,1453.75,1653.75
22,Calaveras,1012.50,1157.50,1446.25,1678.75,1910.00
27,Colusa,838.75,958.75,1197.50,1390.00,1581.25
32,Contra Costa,1568.75,1792.50,2240.00,2598.75,2957.50
37,Del Norte,838.75,958.75,1197.50,1390.00,1581.25
42,El Dorado,1122.50,1282.50,1602.50,1860.00,2116.25
47,Fresno,838.75,958.75,1197.50,1390.00,1581.25


In [33]:
mi_df = df[df.Income_Category =="Median Income"].copy()
mi_df["1"]=mi_df["1"]*0.025
mi_df["2"]=mi_df["2"]*0.025
mi_df["4"]=mi_df["4"]*0.025
mi_df["6"]=mi_df["6"]*0.025
mi_df["8"]=mi_df["8"]*0.025
mi_df.rename(columns={"1":"MI_studio","2":"MI_1_br","4":"MI_2_br","6":"MI_3_br","8":"MI_4_br"}, inplace=True)
mi_df.drop('Income_Category', axis=1, inplace=True)
mi_df

,County,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br
3,Alameda,1827.50,2087.50,2610.00,3027.50,3445.00
8,Alpine,1661.25,1897.50,2372.50,2752.50,3131.25
13,Amador,1287.50,1472.50,1840.00,2135.00,2428.75
18,Butte,1095.00,1252.50,1565.00,1815.00,2066.25
23,Calaveras,1265.00,1446.25,1807.50,2096.25,2386.25
28,Colusa,1048.75,1197.50,1497.50,1737.50,1976.25
33,Contra Costa,1827.50,2087.50,2610.00,3027.50,3445.00
38,Del Norte,1048.75,1197.50,1497.50,1737.50,1976.25
43,El Dorado,1401.25,1602.50,2002.50,2322.50,2643.75
48,Fresno,1048.75,1197.50,1497.50,1737.50,1976.25


In [34]:
moi_df = df[df.Income_Category =="Moderate Income"].copy()
moi_df["1"]=moi_df["1"]*0.025
moi_df["2"]=moi_df["2"]*0.025
moi_df["4"]=moi_df["4"]*0.025
moi_df["6"]=moi_df["6"]*0.025
moi_df["8"]=moi_df["8"]*0.025
moi_df.rename(columns={"1":"MoI_studio","2":"MoI_1_br","4":"MoI_2_br","6":"MoI_3_br","8":"MoI_4_br"}, inplace=True)
moi_df.drop('Income_Category', axis=1, inplace=True)
moi_df

,County,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br
4,Alameda,2192.50,2506.25,3132.50,3633.75,4135.00
9,Alpine,1993.75,2277.50,2847.50,3302.50,3758.75
14,Amador,1545.00,1766.25,2207.50,2561.25,2913.75
19,Butte,1313.75,1502.50,1877.50,2177.50,2478.75
24,Calaveras,1517.50,1735.00,2168.75,2516.25,2862.50
29,Colusa,1258.75,1437.50,1797.50,2085.00,2372.50
34,Contra Costa,2192.50,2506.25,3132.50,3633.75,4135.00
39,Del Norte,2192.50,2506.25,3132.50,3633.75,4135.00
44,El Dorado,1681.25,1922.50,2402.50,2787.50,3171.25
49,Fresno,1258.75,1437.50,1797.50,2085.00,2372.50


In [35]:
#combine dfs
all_amis_df=eli_df.merge(vli_df, how='left', left_on = "County",right_on = "County")
all_amis_df=all_amis_df.merge(li_df, how='left', left_on = "County",right_on = "County")
all_amis_df=all_amis_df.merge(mi_df, how='left', left_on = "County",right_on = "County")
all_amis_df=all_amis_df.merge(moi_df, how='left', left_on = "County",right_on = "County")
all_amis_df

,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br
0,Alameda,610.00,697.50,871.25,1011.25,1151.25,1017.50,1162.50,1452.50,1685.00,1917.50,1568.75,1792.50,2240.00,2598.75,2957.50,1827.50,2087.50,2610.00,3027.50,3445.00,2192.50,2506.25,3132.50,3633.75,4135.00
1,Alpine,453.75,518.75,647.50,843.50,1059.50,756.25,865.00,1080.00,1253.75,1426.25,1152.50,1316.25,1645.00,1908.75,2172.50,1661.25,1897.50,2372.50,2752.50,3131.25,1993.75,2277.50,2847.50,3302.50,3758.75
2,Amador,387.50,442.50,627.50,843.50,1059.50,645.00,736.25,920.00,1067.50,1215.00,1031.25,1178.75,1472.50,1708.75,1943.75,1287.50,1472.50,1840.00,2135.00,2428.75,1545.00,1766.25,2207.50,2561.25,2913.75
3,Butte,330.00,411.50,627.50,843.50,1033.00,548.75,626.25,782.50,908.75,1033.75,877.50,1002.50,1252.50,1453.75,1653.75,1095.00,1252.50,1565.00,1815.00,2066.25,1313.75,1502.50,1877.50,2177.50,2478.75
4,Calaveras,380.00,435.00,627.50,843.50,1059.50,633.75,723.75,903.75,1048.75,1193.75,1012.50,1157.50,1446.25,1678.75,1910.00,1265.00,1446.25,1807.50,2096.25,2386.25,1517.50,1735.00,2168.75,2516.25,2862.50
5,Colusa,315.00,411.50,627.50,843.50,988.75,525.00,600.00,748.75,868.75,988.75,838.75,958.75,1197.50,1390.00,1581.25,1048.75,1197.50,1497.50,1737.50,1976.25,1258.75,1437.50,1797.50,2085.00,2372.50
6,Contra Costa,610.00,697.50,871.25,1011.25,1151.25,1017.50,1162.50,1452.50,1685.00,1917.50,1568.75,1792.50,2240.00,2598.75,2957.50,1827.50,2087.50,2610.00,3027.50,3445.00,2192.50,2506.25,3132.50,3633.75,4135.00
7,Del Norte,315.00,411.50,627.50,843.50,988.75,525.00,600.00,748.75,868.75,988.75,838.75,958.75,1197.50,1390.00,1581.25,1048.75,1197.50,1497.50,1737.50,1976.25,2192.50,2506.25,3132.50,3633.75,4135.00
8,El Dorado,421.25,481.25,627.50,843.50,1059.50,701.25,801.25,1001.25,1162.50,1322.50,1122.50,1282.50,1602.50,1860.00,2116.25,1401.25,1602.50,2002.50,2322.50,2643.75,1681.25,1922.50,2402.50,2787.50,3171.25
9,Fresno,315.00,411.50,627.50,843.50,988.75,525.00,600.00,748.75,868.75,988.75,838.75,958.75,1197.50,1390.00,1581.25,1048.75,1197.50,1497.50,1737.50,1976.25,1258.75,1437.50,1797.50,2085.00,2372.50


## Part A4. Add count dummies for each unit based on income and unit type

In [36]:
# merge datasets
units_df=puma_county_df.merge(all_amis_df, how='left', left_on = "County",right_on = "County")
units_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br
0,H,3701,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,508.75,581.25,726.25,843.50,1059.50,848.75,970.00,1211.25,1406.25,1600.00,1356.25,1550.00,1937.50,2247.50,2557.50,1212.50,1386.25,1732.50,2010.00,2287.50,1455.00,1662.50,2078.75,2411.25,2743.75
1,H,7306,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,San Diego CA,NaN,NaN,NaN,NaN,NaN,NaN,San Diego,511.25,585.00,730.00,847.50,1059.50,852.50,973.75,1216.25,1411.25,1606.25,1362.50,1557.50,1946.25,2258.75,2570.00,1431.25,1636.25,2045.00,2372.50,2700.00,1717.50,1962.50,2453.75,2846.25,3238.75
2,H,3755,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,508.75,581.25,726.25,843.50,1059.50,848.75,970.00,1211.25,1406.25,1600.00,1356.25,1550.00,1937.50,2247.50,2557.50,1212.50,1386.25,1732.50,2010.00,2287.50,1455.00,1662.50,2078.75,2411.25,2743.75
3,H,7319,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,San Diego CA,NaN,NaN,NaN,NaN,NaN,NaN,San Diego,511.25,585.00,730.00,847.50,1059.50,852.50,973.75,1216.25,1411.25,1606.25,1362.50,1557.50,1946.25,2258.75,2570.00,1431.25,1636.25,2045.00,2372.50,2700.00,1717.50,1962.50,2453.75,2846.25,3238.75
4,H,6511,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Riverside CA,NaN,NaN,NaN,NaN,NaN,NaN,Riverside,353.75,411.50,627.50,843.50,1059.50,590.00,675.00,842.50,977.50,1112.50,943.75,1078.75,1347.50,1563.75,1778.75,1151.25,1316.25,1645.00,1908.75,2171.25,1381.25,1578.75,1973.75,2290.00,2605.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159680,H,8103,64.00,4.00,2.00,0.00,nan,nan,nan,2.00,nan,900000.00,nan,nan,78000.00,2.00,11500.00,San Mateo CA,NaN,NaN,NaN,NaN,NaN,NaN,San Mateo,770.00,880.00,1100.00,1276.25,1452.50,1283.75,1466.25,1832.50,2126.25,2420.00,2055.00,2348.75,2935.00,3405.00,3875.00,2072.50,2367.50,2960.00,3433.75,3907.50,2486.25,2842.50,3552.50,4121.25,4688.75
159681,H,5904,121.00,4.00,2.00,0.00,1700.00,nan,nan,1.00,nan,750000.00,nan,nan,105000.00,6.00,3150.00,Orange CA,NaN,NaN,NaN,NaN,NaN,NaN,Orange,575.00,656.25,820.00,951.25,1082.50,957.50,1093.75,1366.25,1585.00,1803.75,1531.25,1750.00,2186.25,2536.25,2886.25,1622.50,1853.75,2317.50,2688.75,3058.75,1947.50,2225.00,2781.25,3226.25,3671.25
159682,H,1304,73.00,4.00,2.00,0.00,1200.00,nan,nan,1.00,nan,2211000.00,nan,nan,118900.00,6.00,3450.00,Contra Costa CA,NaN,NaN,NaN,NaN,NaN,NaN,Contra Costa,610.00,697.50,871.25,1011.25,1151.25,1017.50,1162.50,1452.50,1685.00,1917.50,1568.75,1792.50,2240.00,2598.75,2957.50,1827.50,2087.50,2610.00,3027.50,3445.00,2192.50,2506.25,3132.50,3633.75,4135.00
159683,H,1308,78.00,4.00,2.00,0.00,nan,nan,nan,2.00,nan,600000.00,nan,nan,32400.00,4.00,6500.00,Contra Costa CA,NaN,NaN,NaN,NaN,NaN,NaN,Contra Costa,610.00,697.50,871.25,1011.25,1151.25,1017.50,1162.50,1452.50,1685.00,1917.50,1568.75,1792.50,2240.00,2598.75,2957.50,1827.50,2087.50,2610.00,3027.50,3445.00,2192.50,2506.25,3132.50,3633.75,4135.00


In [37]:
# create filter masks for each affordability and unit type
ELI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['monthly_rent']<=units_df["ELI_studio"])
ELI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['monthly_rent']<=units_df["ELI_1_br"])
ELI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['monthly_rent']<=units_df["ELI_2_br"])
ELI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['monthly_rent']<=units_df["ELI_3_br"])
ELI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['monthly_rent']<=units_df["ELI_4_br"])
VLI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['monthly_rent']<=units_df["VLI_studio"])
VLI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['monthly_rent']<=units_df["VLI_1_br"])
VLI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['monthly_rent']<=units_df["VLI_2_br"])
VLI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['monthly_rent']<=units_df["VLI_3_br"])
VLI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['monthly_rent']<=units_df["VLI_4_br"])
LI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['monthly_rent']<=units_df["LI_studio"])
LI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['monthly_rent']<=units_df["LI_1_br"])
LI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['monthly_rent']<=units_df["LI_2_br"])
LI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['monthly_rent']<=units_df["LI_3_br"])
LI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['monthly_rent']<=units_df["LI_4_br"])
MI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['monthly_rent']<=units_df["MI_studio"])
MI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['monthly_rent']<=units_df["MI_1_br"])
MI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['monthly_rent']<=units_df["MI_2_br"])
MI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['monthly_rent']<=units_df["MI_3_br"])
MI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['monthly_rent']<=units_df["MI_4_br"])
MoI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['monthly_rent']<=units_df["MoI_studio"])
MoI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['monthly_rent']<=units_df["MoI_1_br"])
MoI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['monthly_rent']<=units_df["MoI_2_br"])
MoI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['monthly_rent']<=units_df["MoI_3_br"])
MoI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['monthly_rent']<=units_df["MoI_4_br"])
HI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['monthly_rent']>=units_df["MoI_studio"])
HI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['monthly_rent']>=units_df["MoI_1_br"])
HI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['monthly_rent']>=units_df["MoI_2_br"])
HI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['monthly_rent']>=units_df["MoI_3_br"])
HI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['monthly_rent']>=units_df["MoI_4_br"])

# apply filter masks to create dummy variables
units_df['rent_ELI_studio_count']=np.where(ELI_studio_mask,1,0)
units_df['rent_ELI_1_br_count']=np.where(ELI_1_br_mask,1,0)
units_df['rent_ELI_2_br_count']=np.where(ELI_2_br_mask,1,0)
units_df['rent_ELI_3_br_count']=np.where(ELI_3_br_mask,1,0)
units_df['rent_ELI_4_br_count']=np.where(ELI_4_br_mask,1,0)
units_df['rent_VLI_studio_count']=np.where(VLI_studio_mask,1,0)
units_df['rent_VLI_1_br_count']=np.where(VLI_1_br_mask,1,0)
units_df['rent_VLI_2_br_count']=np.where(VLI_2_br_mask,1,0)
units_df['rent_VLI_3_br_count']=np.where(VLI_3_br_mask,1,0)
units_df['rent_VLI_4_br_count']=np.where(VLI_4_br_mask,1,0)
units_df['rent_LI_studio_count']=np.where(LI_studio_mask,1,0)
units_df['rent_LI_1_br_count']=np.where(LI_1_br_mask,1,0)
units_df['rent_LI_2_br_count']=np.where(LI_2_br_mask,1,0)
units_df['rent_LI_3_br_count']=np.where(LI_3_br_mask,1,0)
units_df['rent_LI_4_br_count']=np.where(LI_4_br_mask,1,0)
units_df['rent_MI_studio_count']=np.where(MI_studio_mask,1,0)
units_df['rent_MI_1_br_count']=np.where(MI_1_br_mask,1,0)
units_df['rent_MI_2_br_count']=np.where(MI_2_br_mask,1,0)
units_df['rent_MI_3_br_count']=np.where(MI_3_br_mask,1,0)
units_df['rent_MI_4_br_count']=np.where(MI_4_br_mask,1,0)
units_df['rent_MoI_studio_count']=np.where(MoI_studio_mask,1,0)
units_df['rent_MoI_1_br_count']=np.where(MoI_1_br_mask,1,0)
units_df['rent_MoI_2_br_count']=np.where(MoI_2_br_mask,1,0)
units_df['rent_MoI_3_br_count']=np.where(MoI_3_br_mask,1,0)
units_df['rent_MoI_4_br_count']=np.where(MoI_4_br_mask,1,0)
units_df['rent_HI_studio_count']=np.where(HI_studio_mask,1,0)
units_df['rent_HI_1_br_count']=np.where(HI_1_br_mask,1,0)
units_df['rent_HI_2_br_count']=np.where(HI_2_br_mask,1,0)
units_df['rent_HI_3_br_count']=np.where(HI_3_br_mask,1,0)
units_df['rent_HI_4_br_count']=np.where(HI_4_br_mask,1,0)
units_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br,rent_ELI_studio_count,rent_ELI_1_br_count,rent_ELI_2_br_count,rent_ELI_3_br_count,rent_ELI_4_br_count,rent_VLI_studio_count,rent_VLI_1_br_count,rent_VLI_2_br_count,rent_VLI_3_br_count,rent_VLI_4_br_count,rent_LI_studio_count,rent_LI_1_br_count,rent_LI_2_br_count,rent_LI_3_br_count,rent_LI_4_br_count,rent_MI_studio_count,rent_MI_1_br_count,rent_MI_2_br_count,rent_MI_3_br_count,rent_MI_4_br_count,rent_MoI_studio_count,rent_MoI_1_br_count,rent_MoI_2_br_count,rent_MoI_3_br_count,rent_MoI_4_br_count,rent_HI_studio_count,rent_HI_1_br_count,rent_HI_2_br_count,rent_HI_3_br_count,rent_HI_4_br_count
0,H,3701,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,508.75,581.25,726.25,843.50,1059.50,848.75,970.00,1211.25,1406.25,1600.00,1356.25,1550.00,1937.50,2247.50,2557.50,1212.50,1386.25,1732.50,2010.00,2287.50,1455.00,1662.50,2078.75,2411.25,2743.75,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,H,7306,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,San Diego CA,NaN,NaN,NaN,NaN,NaN,NaN,San Diego,511.25,585.00,730.00,847.50,1059.50,852.50,973.75,1216.25,1411.25,1606.25,1362.50,1557.50,1946.25,2258.75,2570.00,1431.25,1636.25,2045.00,2372.50,2700.00,1717.50,1962.50,2453.75,2846.25,3238.75,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,H,3755,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,508.75,581.25,726.25,843.50,1059.50,848.75,970.00,1211.25,1406.25,1600.00,1356.25,1550.00,1937.50,2247.50,2557.50,1212.50,1386.25,1732.50,2010.00,2287.50,1455.00,1662.50,2078.75,2411.25,2743.75,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,H,7319,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,San Diego CA,NaN,NaN,NaN,NaN,NaN,NaN,San Diego,511.25,585.00,730.00,847.50,1059.50,852.50,973.75,1216.25,1411.25,1606.25,1362.50,1557.50,1946.25,2258.75,2570.00,1431.25,1636.25,2045.00,2372.50,2700.00,1717.50,1962.50,2453.75,2846.25,3238.75,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,H,6511,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Riverside CA,NaN,NaN,NaN,NaN,NaN,NaN,Riverside,353.75,411.50,627.50,843.50,1059.50,590.00,675.00,842.50,977.50,1112.50,943.75,1078.75,1347.50,1563.75,1778.75,1151.25,1316.25,1645.00,1908.75,2171.25,1381.25,1578.75,1973.75,2290.00,2605.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159680,H,8103,64.00,4.00,2.00,0.00,nan,nan,nan,2.00,nan,900000.00,nan,nan,78000.00,2.00,11500.00,San Mateo CA,NaN,NaN,NaN,NaN,NaN,NaN,San Mateo,770.00,880.00,1100.00,1276.25,1452.50,1283.75,1466.25,1832.50,2126.25,2420.00,2055.00,2348.75,2935.00,3405.00,3875.00,2072.50,2367.50,2960.00,3433.75,3907.50,2486.25,2842.50,3552.50,4121.25,4688.75,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
159681,H,5904,121.00,4.00,2.00,0.00,1700.00,nan,nan,1.00,nan,750000.00,nan,nan,105000.00,6.00,3150.00,Orange CA,NaN,NaN,NaN,NaN,NaN,NaN,Orange,575.00,656.25,820.00,951.25,1082.50,957.50,1093.75,1366.25,1585.00,1803.75,1531.25,1750.00,2186.25,2536.25,2886.25,1622.50,1853.75,2317.50,2688.75,3058.75,1947.50,2225.00,

## Part B1. Calculate total monthly housing cost by adding monthly first mortgage, second/junior mortgage, and condo fees

In [48]:
units_df[units_df['tenure']==2]

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br,...,rent_LI_4_br_count,rent_MI_studio_count,rent_MI_1_br_count,rent_MI_2_br_count,rent_MI_3_br_count,rent_MI_4_br_count,rent_MoI_studio_count,rent_MoI_1_br_count,rent_MoI_2_br_count,rent_MoI_3_br_count,rent_MoI_4_br_count,rent_HI_studio_count,rent_HI_1_br_count,rent_HI_2_br_count,rent_HI_3_br_count,rent_HI_4_br_count,own_monthly_cost,own_ELI_studio_count,own_ELI_1_br_count,own_ELI_2_br_count,own_ELI_3_br_count,own_ELI_4_br_count,own_VLI_studio_count,own_VLI_1_br_count,own_VLI_2_br_count,own_VLI_3_br_count,own_VLI_4_br_count,own_LI_studio_count,own_LI_1_br_count,own_LI_2_br_count,own_LI_3_br_count,own_LI_4_br_count,own_MI_studio_count,own_MI_1_br_count,own_MI_2_br_count,own_MI_3_br_count,own_MI_4_br_count,own_MoI_studio_count,own_MoI_1_br_count,own_MoI_2_br_count,own_MoI_3_br_count,own_MoI_4_br_count,own_HI_studio_count,own_HI_1_br_count,own_HI_2_br_count,own_HI_3_br_count,own_HI_4_br_count,vacant_for_rent,vacant_for_sale,moved_last_year
16914,H,3764,61.00,4.00,2.00,0.00,nan,nan,nan,2.00,nan,700000.00,nan,nan,24000.00,7.00,3150.00,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,508.75,581.25,726.25,843.50,1059.50,848.75,970.00,1211.25,1406.25,1600.00,1356.25,1550.00,1937.50,2247.50,2557.50,1212.50,1386.25,1732.50,2010.00,2287.50,1455.00,1662.50,2078.75,2411.25,2743.75,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0
16916,H,6505,48.00,2.00,1.00,0.00,nan,nan,nan,2.00,nan,45000.00,nan,nan,31050.00,3.00,0.00,Riverside CA,NaN,NaN,NaN,NaN,NaN,NaN,Riverside,353.75,411.50,627.50,843.50,1059.50,590.00,675.00,842.50,977.50,1112.50,943.75,1078.75,1347.50,1563.75,1778.75,1151.25,1316.25,1645.00,1908.75,2171.25,1381.25,1578.75,1973.75,2290.00,2605.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0
16917,H,3745,69.00,3.00,2.00,0.00,nan,nan,nan,2.00,nan,300000.00,nan,nan,86400.00,7.00,1350.00,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,508.75,581.25,726.25,843.50,1059.50,848.75,970.00,1211.25,1406.25,1600.00,1356.25,1550.00,1937.50,2247.50,2557.50,1212.50,1386.25,1732.50,2010.00,2287.50,1455.00,1662.50,2078.75,2411.25,2743.75,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0
16924,H,7113,60.00,1.00,3.00,310.00,nan,nan,nan,2.00,nan,250000.00,nan,nan,70000.00,3.00,0.00,San Bernardino CA,NaN,NaN,NaN,NaN,NaN,NaN,San Bernardino,353.75,411.50,627.50,843.50,1059.50,590.00,675.00,842.50,977.50,1112.50,943.75,1078.75,1347.50,1563.75,1778.75,1151.25,1316.25,1645.00,1908.75,2171.25,1381.25,1578.75,1973.75,2290.00,2605.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0
16929,H,8512,47.00,6.00,2.00,0.00,nan,nan,nan,2.00,nan,1500000.00,nan,nan,564700.00,7.00,2750.00,Santa Clara CA,NaN,NaN,NaN,NaN,NaN,NaN,Santa Clara,698.75,798.75,997.50,1157.50,1317.50,1163.75,1330.00,1662.50,1928.75,21

In [38]:
units_df['own_monthly_cost']=units_df['first_mortgage'] + units_df['second_mortgage'] + units_df['condo_fee'] + (units_df['prop_tax']/12)
units_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br,rent_ELI_studio_count,rent_ELI_1_br_count,rent_ELI_2_br_count,rent_ELI_3_br_count,rent_ELI_4_br_count,rent_VLI_studio_count,rent_VLI_1_br_count,rent_VLI_2_br_count,rent_VLI_3_br_count,rent_VLI_4_br_count,rent_LI_studio_count,rent_LI_1_br_count,rent_LI_2_br_count,rent_LI_3_br_count,rent_LI_4_br_count,rent_MI_studio_count,rent_MI_1_br_count,rent_MI_2_br_count,rent_MI_3_br_count,rent_MI_4_br_count,rent_MoI_studio_count,rent_MoI_1_br_count,rent_MoI_2_br_count,rent_MoI_3_br_count,rent_MoI_4_br_count,rent_HI_studio_count,rent_HI_1_br_count,rent_HI_2_br_count,rent_HI_3_br_count,rent_HI_4_br_count,own_monthly_cost
0,H,3701,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,508.75,581.25,726.25,843.50,1059.50,848.75,970.00,1211.25,1406.25,1600.00,1356.25,1550.00,1937.50,2247.50,2557.50,1212.50,1386.25,1732.50,2010.00,2287.50,1455.00,1662.50,2078.75,2411.25,2743.75,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nan
1,H,7306,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,San Diego CA,NaN,NaN,NaN,NaN,NaN,NaN,San Diego,511.25,585.00,730.00,847.50,1059.50,852.50,973.75,1216.25,1411.25,1606.25,1362.50,1557.50,1946.25,2258.75,2570.00,1431.25,1636.25,2045.00,2372.50,2700.00,1717.50,1962.50,2453.75,2846.25,3238.75,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nan
2,H,3755,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,508.75,581.25,726.25,843.50,1059.50,848.75,970.00,1211.25,1406.25,1600.00,1356.25,1550.00,1937.50,2247.50,2557.50,1212.50,1386.25,1732.50,2010.00,2287.50,1455.00,1662.50,2078.75,2411.25,2743.75,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nan
3,H,7319,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,San Diego CA,NaN,NaN,NaN,NaN,NaN,NaN,San Diego,511.25,585.00,730.00,847.50,1059.50,852.50,973.75,1216.25,1411.25,1606.25,1362.50,1557.50,1946.25,2258.75,2570.00,1431.25,1636.25,2045.00,2372.50,2700.00,1717.50,1962.50,2453.75,2846.25,3238.75,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nan
4,H,6511,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Riverside CA,NaN,NaN,NaN,NaN,NaN,NaN,Riverside,353.75,411.50,627.50,843.50,1059.50,590.00,675.00,842.50,977.50,1112.50,943.75,1078.75,1347.50,1563.75,1778.75,1151.25,1316.25,1645.00,1908.75,2171.25,1381.25,1578.75,1973.75,2290.00,2605.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159680,H,8103,64.00,4.00,2.00,0.00,nan,nan,nan,2.00,nan,900000.00,nan,nan,78000.00,2.00,11500.00,San Mateo CA,NaN,NaN,NaN,NaN,NaN,NaN,San Mateo,770.00,880.00,1100.00,1276.25,1452.50,1283.75,1466.25,1832.50,2126.25,2420.00,2055.00,2348.75,2935.00,3405.00,3875.00,2072.50,2367.50,2960.00,3433.75,3907.50,2486.25,2842.50,3552.50,4121.25,4688.75,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nan
159681,H,5904,121.00,4.00,2.00,0.00,1700.00,nan,nan,1.00,nan,750000.00,nan,nan,105000.00,6.00,3150.00,Orange CA,NaN,NaN,NaN,NaN,NaN,NaN,Orange,575.00,656.25,820.00,951.25,1082.50,957.50,1093.75,1366.25,1585.00,1803.75,1531.25,1750.00,2186.25,2536.25,2886.25,1622.50,185

# Part B2. Add count dummies for each unit based on income and unit type

In [39]:
# create filter masks for each affordability and unit type
ELI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['own_monthly_cost']<=units_df["ELI_studio"])
ELI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['own_monthly_cost']<=units_df["ELI_1_br"])
ELI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['own_monthly_cost']<=units_df["ELI_2_br"])
ELI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['own_monthly_cost']<=units_df["ELI_3_br"])
ELI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['own_monthly_cost']<=units_df["ELI_4_br"])
VLI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['own_monthly_cost']<=units_df["VLI_studio"])
VLI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['own_monthly_cost']<=units_df["VLI_1_br"])
VLI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['own_monthly_cost']<=units_df["VLI_2_br"])
VLI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['own_monthly_cost']<=units_df["VLI_3_br"])
VLI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['own_monthly_cost']<=units_df["VLI_4_br"])
LI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['own_monthly_cost']<=units_df["LI_studio"])
LI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['own_monthly_cost']<=units_df["LI_1_br"])
LI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['own_monthly_cost']<=units_df["LI_2_br"])
LI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['own_monthly_cost']<=units_df["LI_3_br"])
LI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['own_monthly_cost']<=units_df["LI_4_br"])
MI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['own_monthly_cost']<=units_df["MI_studio"])
MI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['own_monthly_cost']<=units_df["MI_1_br"])
MI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['own_monthly_cost']<=units_df["MI_2_br"])
MI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['own_monthly_cost']<=units_df["MI_3_br"])
MI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['own_monthly_cost']<=units_df["MI_4_br"])
MoI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['own_monthly_cost']<=units_df["MoI_studio"])
MoI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['own_monthly_cost']<=units_df["MoI_1_br"])
MoI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['own_monthly_cost']<=units_df["MoI_2_br"])
MoI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['own_monthly_cost']<=units_df["MoI_3_br"])
MoI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['own_monthly_cost']<=units_df["MoI_4_br"])
HI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['own_monthly_cost']>=units_df["MoI_studio"])
HI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['own_monthly_cost']>=units_df["MoI_1_br"])
HI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['own_monthly_cost']>=units_df["MoI_2_br"])
HI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['own_monthly_cost']>=units_df["MoI_3_br"])
HI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['own_monthly_cost']>=units_df["MoI_4_br"])

# apply filter masks to create dummy variables
units_df['own_ELI_studio_count']=np.where(ELI_studio_mask,1,0)
units_df['own_ELI_1_br_count']=np.where(ELI_1_br_mask,1,0)
units_df['own_ELI_2_br_count']=np.where(ELI_2_br_mask,1,0)
units_df['own_ELI_3_br_count']=np.where(ELI_3_br_mask,1,0)
units_df['own_ELI_4_br_count']=np.where(ELI_4_br_mask,1,0)
units_df['own_VLI_studio_count']=np.where(VLI_studio_mask,1,0)
units_df['own_VLI_1_br_count']=np.where(VLI_1_br_mask,1,0)
units_df['own_VLI_2_br_count']=np.where(VLI_2_br_mask,1,0)
units_df['own_VLI_3_br_count']=np.where(VLI_3_br_mask,1,0)
units_df['own_VLI_4_br_count']=np.where(VLI_4_br_mask,1,0)
units_df['own_LI_studio_count']=np.where(LI_studio_mask,1,0)
units_df['own_LI_1_br_count']=np.where(LI_1_br_mask,1,0)
units_df['own_LI_2_br_count']=np.where(LI_2_br_mask,1,0)
units_df['own_LI_3_br_count']=np.where(LI_3_br_mask,1,0)
units_df['own_LI_4_br_count']=np.where(LI_4_br_mask,1,0)
units_df['own_MI_studio_count']=np.where(MI_studio_mask,1,0)
units_df['own_MI_1_br_count']=np.where(MI_1_br_mask,1,0)
units_df['own_MI_2_br_count']=np.where(MI_2_br_mask,1,0)
units_df['own_MI_3_br_count']=np.where(MI_3_br_mask,1,0)
units_df['own_MI_4_br_count']=np.where(MI_4_br_mask,1,0)
units_df['own_MoI_studio_count']=np.where(MoI_studio_mask,1,0)
units_df['own_MoI_1_br_count']=np.where(MoI_1_br_mask,1,0)
units_df['own_MoI_2_br_count']=np.where(MoI_2_br_mask,1,0)
units_df['own_MoI_3_br_count']=np.where(MoI_3_br_mask,1,0)
units_df['own_MoI_4_br_count']=np.where(MoI_4_br_mask,1,0)
units_df['own_HI_studio_count']=np.where(HI_studio_mask,1,0)
units_df['own_HI_1_br_count']=np.where(HI_1_br_mask,1,0)
units_df['own_HI_2_br_count']=np.where(HI_2_br_mask,1,0)
units_df['own_HI_3_br_count']=np.where(HI_3_br_mask,1,0)
units_df['own_HI_4_br_count']=np.where(HI_4_br_mask,1,0)
units_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br,...,rent_LI_1_br_count,rent_LI_2_br_count,rent_LI_3_br_count,rent_LI_4_br_count,rent_MI_studio_count,rent_MI_1_br_count,rent_MI_2_br_count,rent_MI_3_br_count,rent_MI_4_br_count,rent_MoI_studio_count,rent_MoI_1_br_count,rent_MoI_2_br_count,rent_MoI_3_br_count,rent_MoI_4_br_count,rent_HI_studio_count,rent_HI_1_br_count,rent_HI_2_br_count,rent_HI_3_br_count,rent_HI_4_br_count,own_monthly_cost,own_ELI_studio_count,own_ELI_1_br_count,own_ELI_2_br_count,own_ELI_3_br_count,own_ELI_4_br_count,own_VLI_studio_count,own_VLI_1_br_count,own_VLI_2_br_count,own_VLI_3_br_count,own_VLI_4_br_count,own_LI_studio_count,own_LI_1_br_count,own_LI_2_br_count,own_LI_3_br_count,own_LI_4_br_count,own_MI_studio_count,own_MI_1_br_count,own_MI_2_br_count,own_MI_3_br_count,own_MI_4_br_count,own_MoI_studio_count,own_MoI_1_br_count,own_MoI_2_br_count,own_MoI_3_br_count,own_MoI_4_br_count,own_HI_studio_count,own_HI_1_br_count,own_HI_2_br_count,own_HI_3_br_count,own_HI_4_br_count
0,H,3701,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,508.75,581.25,726.25,843.50,1059.50,848.75,970.00,1211.25,1406.25,1600.00,1356.25,1550.00,1937.50,2247.50,2557.50,1212.50,1386.25,1732.50,2010.00,2287.50,1455.00,1662.50,2078.75,2411.25,2743.75,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,H,7306,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,San Diego CA,NaN,NaN,NaN,NaN,NaN,NaN,San Diego,511.25,585.00,730.00,847.50,1059.50,852.50,973.75,1216.25,1411.25,1606.25,1362.50,1557.50,1946.25,2258.75,2570.00,1431.25,1636.25,2045.00,2372.50,2700.00,1717.50,1962.50,2453.75,2846.25,3238.75,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,H,3755,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,508.75,581.25,726.25,843.50,1059.50,848.75,970.00,1211.25,1406.25,1600.00,1356.25,1550.00,1937.50,2247.50,2557.50,1212.50,1386.25,1732.50,2010.00,2287.50,1455.00,1662.50,2078.75,2411.25,2743.75,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,H,7319,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,San Diego CA,NaN,NaN,NaN,NaN,NaN,NaN,San Diego,511.25,585.00,730.00,847.50,1059.50,852.50,973.75,1216.25,1411.25,1606.25,1362.50,1557.50,1946.25,2258.75,2570.00,1431.25,1636.25,2045.00,2372.50,2700.00,1717.50,1962.50,2453.75,2846.25,3238.75,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,H,6511,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Riverside CA,NaN,NaN,NaN,NaN,NaN,NaN,Riverside,353.75,411.50,627.50,843.50,1059.50,590.00,675.00,842.50,977.50,1112.50,943.75,1078.75,1347.50,1563.75,1778.75,1151.25,1316.25,1645.00,1908.75,2171.25,1381.25,1578.75,1973.75,2290.00,2605.00,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159680,H,8103,64.00,4.00,2.00,0.00,nan,

# Part C1. Multiply each dummy by the the household weight variable

In [40]:
units_df.rent_ELI_studio_count=units_df.rent_ELI_studio_count*units_df.weight
units_df.rent_ELI_1_br_count=units_df.rent_ELI_1_br_count*units_df.weight
units_df.rent_ELI_2_br_count=units_df.rent_ELI_2_br_count*units_df.weight
units_df.rent_ELI_3_br_count=units_df.rent_ELI_3_br_count*units_df.weight
units_df.rent_ELI_4_br_count=units_df.rent_ELI_4_br_count*units_df.weight
units_df.rent_VLI_studio_count=units_df.rent_VLI_studio_count*units_df.weight
units_df.rent_VLI_1_br_count=units_df.rent_VLI_1_br_count*units_df.weight
units_df.rent_VLI_2_br_count=units_df.rent_VLI_2_br_count*units_df.weight
units_df.rent_VLI_3_br_count=units_df.rent_VLI_3_br_count*units_df.weight
units_df.rent_VLI_4_br_count=units_df.rent_VLI_4_br_count*units_df.weight
units_df.rent_LI_studio_count=units_df.rent_LI_studio_count*units_df.weight
units_df.rent_LI_1_br_count=units_df.rent_LI_1_br_count*units_df.weight
units_df.rent_LI_2_br_count=units_df.rent_LI_2_br_count*units_df.weight
units_df.rent_LI_3_br_count=units_df.rent_LI_3_br_count*units_df.weight
units_df.rent_LI_4_br_count=units_df.rent_LI_4_br_count*units_df.weight
units_df.rent_MI_studio_count=units_df.rent_MI_studio_count*units_df.weight
units_df.rent_MI_1_br_count=units_df.rent_MI_1_br_count*units_df.weight
units_df.rent_MI_2_br_count=units_df.rent_MI_2_br_count*units_df.weight
units_df.rent_MI_3_br_count=units_df.rent_MI_3_br_count*units_df.weight
units_df.rent_MI_4_br_count=units_df.rent_MI_4_br_count*units_df.weight
units_df.rent_MoI_studio_count=units_df.rent_MoI_studio_count*units_df.weight
units_df.rent_MoI_1_br_count=units_df.rent_MoI_1_br_count*units_df.weight
units_df.rent_MoI_2_br_count=units_df.rent_MoI_2_br_count*units_df.weight
units_df.rent_MoI_3_br_count=units_df.rent_MoI_3_br_count*units_df.weight
units_df.rent_MoI_4_br_count=units_df.rent_MoI_4_br_count*units_df.weight
units_df.rent_HI_studio_count=units_df.rent_HI_studio_count*units_df.weight
units_df.rent_HI_1_br_count=units_df.rent_HI_1_br_count*units_df.weight
units_df.rent_HI_2_br_count=units_df.rent_HI_2_br_count*units_df.weight
units_df.rent_HI_3_br_count=units_df.rent_HI_3_br_count*units_df.weight
units_df.rent_HI_4_br_count=units_df.rent_HI_4_br_count*units_df.weight
units_df.own_ELI_studio_count=units_df.own_ELI_studio_count*units_df.weight
units_df.own_ELI_1_br_count=units_df.own_ELI_1_br_count*units_df.weight
units_df.own_ELI_2_br_count=units_df.own_ELI_2_br_count*units_df.weight
units_df.own_ELI_3_br_count=units_df.own_ELI_3_br_count*units_df.weight
units_df.own_ELI_4_br_count=units_df.own_ELI_4_br_count*units_df.weight
units_df.own_VLI_studio_count=units_df.own_VLI_studio_count*units_df.weight
units_df.own_VLI_1_br_count=units_df.own_VLI_1_br_count*units_df.weight
units_df.own_VLI_2_br_count=units_df.own_VLI_2_br_count*units_df.weight
units_df.own_VLI_3_br_count=units_df.own_VLI_3_br_count*units_df.weight
units_df.own_VLI_4_br_count=units_df.own_VLI_4_br_count*units_df.weight
units_df.own_LI_studio_count=units_df.own_LI_studio_count*units_df.weight
units_df.own_LI_1_br_count=units_df.own_LI_1_br_count*units_df.weight
units_df.own_LI_2_br_count=units_df.own_LI_2_br_count*units_df.weight
units_df.own_LI_3_br_count=units_df.own_LI_3_br_count*units_df.weight
units_df.own_LI_4_br_count=units_df.own_LI_4_br_count*units_df.weight
units_df.own_MI_studio_count=units_df.own_MI_studio_count*units_df.weight
units_df.own_MI_1_br_count=units_df.own_MI_1_br_count*units_df.weight
units_df.own_MI_2_br_count=units_df.own_MI_2_br_count*units_df.weight
units_df.own_MI_3_br_count=units_df.own_MI_3_br_count*units_df.weight
units_df.own_MI_4_br_count=units_df.own_MI_4_br_count*units_df.weight
units_df.own_MoI_studio_count=units_df.own_MoI_studio_count*units_df.weight
units_df.own_MoI_1_br_count=units_df.own_MoI_1_br_count*units_df.weight
units_df.own_MoI_2_br_count=units_df.own_MoI_2_br_count*units_df.weight
units_df.own_MoI_3_br_count=units_df.own_MoI_3_br_count*units_df.weight
units_df.own_MoI_4_br_count=units_df.own_MoI_4_br_count*units_df.weight
units_df.own_HI_studio_count=units_df.own_HI_studio_count*units_df.weight
units_df.own_HI_1_br_count=units_df.own_HI_1_br_count*units_df.weight
units_df.own_HI_2_br_count=units_df.own_HI_2_br_count*units_df.weight
units_df.own_HI_3_br_count=units_df.own_HI_3_br_count*units_df.weight
units_df.own_HI_4_br_count=units_df.own_HI_4_br_count*units_df.weight
units_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br,...,rent_LI_1_br_count,rent_LI_2_br_count,rent_LI_3_br_count,rent_LI_4_br_count,rent_MI_studio_count,rent_MI_1_br_count,rent_MI_2_br_count,rent_MI_3_br_count,rent_MI_4_br_count,rent_MoI_studio_count,rent_MoI_1_br_count,rent_MoI_2_br_count,rent_MoI_3_br_count,rent_MoI_4_br_count,rent_HI_studio_count,rent_HI_1_br_count,rent_HI_2_br_count,rent_HI_3_br_count,rent_HI_4_br_count,own_monthly_cost,own_ELI_studio_count,own_ELI_1_br_count,own_ELI_2_br_count,own_ELI_3_br_count,own_ELI_4_br_count,own_VLI_studio_count,own_VLI_1_br_count,own_VLI_2_br_count,own_VLI_3_br_count,own_VLI_4_br_count,own_LI_studio_count,own_LI_1_br_count,own_LI_2_br_count,own_LI_3_br_count,own_LI_4_br_count,own_MI_studio_count,own_MI_1_br_count,own_MI_2_br_count,own_MI_3_br_count,own_MI_4_br_count,own_MoI_studio_count,own_MoI_1_br_count,own_MoI_2_br_count,own_MoI_3_br_count,own_MoI_4_br_count,own_HI_studio_count,own_HI_1_br_count,own_HI_2_br_count,own_HI_3_br_count,own_HI_4_br_count
0,H,3701,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,508.75,581.25,726.25,843.50,1059.50,848.75,970.00,1211.25,1406.25,1600.00,1356.25,1550.00,1937.50,2247.50,2557.50,1212.50,1386.25,1732.50,2010.00,2287.50,1455.00,1662.50,2078.75,2411.25,2743.75,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,H,7306,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,San Diego CA,NaN,NaN,NaN,NaN,NaN,NaN,San Diego,511.25,585.00,730.00,847.50,1059.50,852.50,973.75,1216.25,1411.25,1606.25,1362.50,1557.50,1946.25,2258.75,2570.00,1431.25,1636.25,2045.00,2372.50,2700.00,1717.50,1962.50,2453.75,2846.25,3238.75,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,H,3755,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,508.75,581.25,726.25,843.50,1059.50,848.75,970.00,1211.25,1406.25,1600.00,1356.25,1550.00,1937.50,2247.50,2557.50,1212.50,1386.25,1732.50,2010.00,2287.50,1455.00,1662.50,2078.75,2411.25,2743.75,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,H,7319,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,San Diego CA,NaN,NaN,NaN,NaN,NaN,NaN,San Diego,511.25,585.00,730.00,847.50,1059.50,852.50,973.75,1216.25,1411.25,1606.25,1362.50,1557.50,1946.25,2258.75,2570.00,1431.25,1636.25,2045.00,2372.50,2700.00,1717.50,1962.50,2453.75,2846.25,3238.75,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,H,6511,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Riverside CA,NaN,NaN,NaN,NaN,NaN,NaN,Riverside,353.75,411.50,627.50,843.50,1059.50,590.00,675.00,842.50,977.50,1112.50,943.75,1078.75,1347.50,1563.75,1778.75,1151.25,1316.25,1645.00,1908.75,2171.25,1381.

In [41]:
units_df[units_df.rent_HI_4_br_count>0]

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br,...,rent_LI_1_br_count,rent_LI_2_br_count,rent_LI_3_br_count,rent_LI_4_br_count,rent_MI_studio_count,rent_MI_1_br_count,rent_MI_2_br_count,rent_MI_3_br_count,rent_MI_4_br_count,rent_MoI_studio_count,rent_MoI_1_br_count,rent_MoI_2_br_count,rent_MoI_3_br_count,rent_MoI_4_br_count,rent_HI_studio_count,rent_HI_1_br_count,rent_HI_2_br_count,rent_HI_3_br_count,rent_HI_4_br_count,own_monthly_cost,own_ELI_studio_count,own_ELI_1_br_count,own_ELI_2_br_count,own_ELI_3_br_count,own_ELI_4_br_count,own_VLI_studio_count,own_VLI_1_br_count,own_VLI_2_br_count,own_VLI_3_br_count,own_VLI_4_br_count,own_LI_studio_count,own_LI_1_br_count,own_LI_2_br_count,own_LI_3_br_count,own_LI_4_br_count,own_MI_studio_count,own_MI_1_br_count,own_MI_2_br_count,own_MI_3_br_count,own_MI_4_br_count,own_MoI_studio_count,own_MoI_1_br_count,own_MoI_2_br_count,own_MoI_3_br_count,own_MoI_4_br_count,own_HI_studio_count,own_HI_1_br_count,own_HI_2_br_count,own_HI_3_br_count,own_HI_4_br_count
17348,H,3768,101.00,4.00,2.00,nan,nan,3100.00,nan,3.00,nan,nan,3549.00,21.00,206000.00,4.00,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,508.75,581.25,726.25,843.50,1059.50,848.75,970.00,1211.25,1406.25,1600.00,1356.25,1550.00,1937.50,2247.50,2557.50,1212.50,1386.25,1732.50,2010.00,2287.50,1455.00,1662.50,2078.75,2411.25,2743.75,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,101.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
18030,H,3731,59.00,4.00,2.00,nan,nan,3500.00,nan,3.00,nan,nan,4213.00,15.00,343000.00,2.00,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,508.75,581.25,726.25,843.50,1059.50,848.75,970.00,1211.25,1406.25,1600.00,1356.25,1550.00,1937.50,2247.50,2557.50,1212.50,1386.25,1732.50,2010.00,2287.50,1455.00,1662.50,2078.75,2411.25,2743.75,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,59.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
18355,H,3760,383.00,4.00,2.00,nan,nan,3600.00,nan,3.00,nan,nan,3795.00,18.00,259600.00,4.00,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,508.75,581.25,726.25,843.50,1059.50,848.75,970.00,1211.25,1406.25,1600.00,1356.25,1550.00,1937.50,2247.50,2557.50,1212.50,1386.25,1732.50,2010.00,2287.50,1455.00,1662.50,2078.75,2411.25,2743.75,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,383.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
18564,H,3725,70.00,4.00,2.00,nan,nan,3500.00,nan,3.00,nan,nan,4040.00,48.00,101900.00,3.00,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,508.75,581.25,726.25,843.50,1059.50,848.75,970.00,1211.25,1406.25,1600.00,1356.25,1550.00,1937.50,2247.50,2557.50,1212.50,1386.25,1732.50,2010.00,2287.50,1455.00,1662.50,2078.75,2411.25,2743.75,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,70.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
19791,H,3768,141.00,4.00,2.00,nan,nan,3400.00,nan,3.00,nan,nan,3993.00,23.00,209400.00,1.00,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,508.75,581

# Part D1. Add dummy variables for vacant "for-rent," vacant "for-sale," and recently moved in

In [42]:
# create dummy variables for vacant for rent, vacant for sale, rented in the last year, and sold in the last year
units_df["vacant_for_rent"]=np.where(units_df['vacancy_status']==1,1,0)
units_df["vacant_for_sale"]=np.where(units_df['vacancy_status']==3,1,0)
units_df["moved_last_year"]=np.where(units_df['moved_in']==1,1,0)
units_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br,...,rent_LI_4_br_count,rent_MI_studio_count,rent_MI_1_br_count,rent_MI_2_br_count,rent_MI_3_br_count,rent_MI_4_br_count,rent_MoI_studio_count,rent_MoI_1_br_count,rent_MoI_2_br_count,rent_MoI_3_br_count,rent_MoI_4_br_count,rent_HI_studio_count,rent_HI_1_br_count,rent_HI_2_br_count,rent_HI_3_br_count,rent_HI_4_br_count,own_monthly_cost,own_ELI_studio_count,own_ELI_1_br_count,own_ELI_2_br_count,own_ELI_3_br_count,own_ELI_4_br_count,own_VLI_studio_count,own_VLI_1_br_count,own_VLI_2_br_count,own_VLI_3_br_count,own_VLI_4_br_count,own_LI_studio_count,own_LI_1_br_count,own_LI_2_br_count,own_LI_3_br_count,own_LI_4_br_count,own_MI_studio_count,own_MI_1_br_count,own_MI_2_br_count,own_MI_3_br_count,own_MI_4_br_count,own_MoI_studio_count,own_MoI_1_br_count,own_MoI_2_br_count,own_MoI_3_br_count,own_MoI_4_br_count,own_HI_studio_count,own_HI_1_br_count,own_HI_2_br_count,own_HI_3_br_count,own_HI_4_br_count,vacant_for_rent,vacant_for_sale,moved_last_year
0,H,3701,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,508.75,581.25,726.25,843.50,1059.50,848.75,970.00,1211.25,1406.25,1600.00,1356.25,1550.00,1937.50,2247.50,2557.50,1212.50,1386.25,1732.50,2010.00,2287.50,1455.00,1662.50,2078.75,2411.25,2743.75,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0
1,H,7306,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,San Diego CA,NaN,NaN,NaN,NaN,NaN,NaN,San Diego,511.25,585.00,730.00,847.50,1059.50,852.50,973.75,1216.25,1411.25,1606.25,1362.50,1557.50,1946.25,2258.75,2570.00,1431.25,1636.25,2045.00,2372.50,2700.00,1717.50,1962.50,2453.75,2846.25,3238.75,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0
2,H,3755,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,508.75,581.25,726.25,843.50,1059.50,848.75,970.00,1211.25,1406.25,1600.00,1356.25,1550.00,1937.50,2247.50,2557.50,1212.50,1386.25,1732.50,2010.00,2287.50,1455.00,1662.50,2078.75,2411.25,2743.75,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0
3,H,7319,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,San Diego CA,NaN,NaN,NaN,NaN,NaN,NaN,San Diego,511.25,585.00,730.00,847.50,1059.50,852.50,973.75,1216.25,1411.25,1606.25,1362.50,1557.50,1946.25,2258.75,2570.00,1431.25,1636.25,2045.00,2372.50,2700.00,1717.50,1962.50,2453.75,2846.25,3238.75,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0
4,H,6511,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Riverside CA,NaN,NaN,NaN,NaN,NaN,NaN,Riverside,353.75,411.50,627.50,843.50,1059.50,590.00,675.00,842.50,977.50,1112.50,943.75,1078.75,1347.50,1563.75,1778.75,1151.25,1316.25,1645.00,1908.75,2171.25,1381.25,1578.75,1973.75,2290.00,2605.00,...,0.00,0

In [43]:
#filter for only currently listed or recently moved into units
recent_mask = (units_df.vacant_for_rent==1)|(units_df.vacant_for_sale==1)|(units_df.moved_last_year==1)
recent_units_df = units_df[recent_mask]
recent_units_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br,...,rent_LI_4_br_count,rent_MI_studio_count,rent_MI_1_br_count,rent_MI_2_br_count,rent_MI_3_br_count,rent_MI_4_br_count,rent_MoI_studio_count,rent_MoI_1_br_count,rent_MoI_2_br_count,rent_MoI_3_br_count,rent_MoI_4_br_count,rent_HI_studio_count,rent_HI_1_br_count,rent_HI_2_br_count,rent_HI_3_br_count,rent_HI_4_br_count,own_monthly_cost,own_ELI_studio_count,own_ELI_1_br_count,own_ELI_2_br_count,own_ELI_3_br_count,own_ELI_4_br_count,own_VLI_studio_count,own_VLI_1_br_count,own_VLI_2_br_count,own_VLI_3_br_count,own_VLI_4_br_count,own_LI_studio_count,own_LI_1_br_count,own_LI_2_br_count,own_LI_3_br_count,own_LI_4_br_count,own_MI_studio_count,own_MI_1_br_count,own_MI_2_br_count,own_MI_3_br_count,own_MI_4_br_count,own_MoI_studio_count,own_MoI_1_br_count,own_MoI_2_br_count,own_MoI_3_br_count,own_MoI_4_br_count,own_HI_studio_count,own_HI_1_br_count,own_HI_2_br_count,own_HI_3_br_count,own_HI_4_br_count,vacant_for_rent,vacant_for_sale,moved_last_year
16915,H,7504,110.00,2.00,6.00,nan,nan,3900.00,nan,3.00,nan,nan,3900.00,18.00,266500.00,1.00,nan,San Francisco CA,NaN,NaN,NaN,NaN,NaN,NaN,San Francisco,770.00,880.00,1100.00,1276.25,1452.50,1283.75,1466.25,1832.50,2126.25,2420.00,2055.00,2348.75,2935.00,3405.00,3875.00,2072.50,2367.50,2960.00,3433.75,3907.50,2486.25,2842.50,3552.50,4121.25,4688.75,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,110.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,1
16969,H,5914,378.00,3.00,6.00,0.00,7300.00,nan,nan,1.00,nan,1150000.00,nan,nan,120000.00,1.00,12500.00,Orange CA,NaN,NaN,NaN,NaN,NaN,NaN,Orange,575.00,656.25,820.00,951.25,1082.50,957.50,1093.75,1366.25,1585.00,1803.75,1531.25,1750.00,2186.25,2536.25,2886.25,1622.50,1853.75,2317.50,2688.75,3058.75,1947.50,2225.00,2781.25,3226.25,3671.25,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,1
16971,H,2903,160.00,3.00,4.00,nan,nan,190.00,nan,3.00,nan,nan,390.00,101.00,3600.00,1.00,nan,Kern CA,NaN,NaN,NaN,NaN,NaN,NaN,Kern,315.00,411.50,627.50,843.50,988.75,525.00,600.00,748.75,868.75,988.75,838.75,958.75,1197.50,1390.00,1581.25,1048.75,1197.50,1497.50,1737.50,1976.25,1258.75,1437.50,1797.50,2085.00,2372.50,...,0.00,0.00,0.00,0.00,160.00,0.00,0.00,0.00,0.00,160.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,1
16988,H,3750,81.00,4.00,3.00,0.00,nan,nan,nan,2.00,nan,47500.00,nan,nan,165900.00,1.00,0.00,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,508.75,581.25,726.25,843.50,1059.50,848.75,970.00,1211.25,1406.25,1600.00,1356.25,1550.00,1937.50,2247.50,2557.50,1212.50,1386.25,1732.50,2010.00,2287.50,1455.00,1662.50,2078.75,2411.25,2743.75,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,1
16995,H,8505,76.00,4.00,2.00,0.00,nan,nan,nan,2.00,nan,1151000.00,nan,nan,148500.00,1.00,9500.00,Santa Clara CA,NaN,NaN,NaN,NaN,NaN,NaN,Santa Clara,698.75,798.75,997.50,1157.50,1317.50,1163.75,1330.00,1662.50,1928.75,2195

# Part D2. Aggregating at the PUMA level

In [44]:
# for all units
puma_sums_df = units_df.groupby("PUMA").sum()
puma_counts_df = puma_sums_df[["rent_ELI_studio_count","rent_ELI_1_br_count",
    "rent_ELI_2_br_count",
    "rent_ELI_3_br_count",
    "rent_ELI_4_br_count",
    "rent_VLI_studio_count",
    "rent_VLI_1_br_count",
    "rent_VLI_2_br_count",
    "rent_VLI_3_br_count",
    "rent_VLI_4_br_count",
    "rent_LI_studio_count",
    "rent_LI_1_br_count",
    "rent_LI_2_br_count",
    "rent_LI_3_br_count",
    "rent_LI_4_br_count",
    "rent_MI_studio_count",
    "rent_MI_1_br_count",
    "rent_MI_2_br_count",
    "rent_MI_3_br_count",
    "rent_MI_4_br_count",
    "rent_MoI_studio_count",
    "rent_MoI_1_br_count",
    "rent_MoI_2_br_count",
    "rent_MoI_3_br_count",
    "rent_MoI_4_br_count",
    "rent_HI_studio_count",
    "rent_HI_1_br_count",
    "rent_HI_2_br_count",
    "rent_HI_3_br_count",
    "rent_HI_4_br_count",
    "own_ELI_studio_count",
    "own_ELI_1_br_count",
    "own_ELI_2_br_count",
    "own_ELI_3_br_count",
    "own_ELI_4_br_count",
    "own_VLI_studio_count",
    "own_VLI_1_br_count",
    "own_VLI_2_br_count",
    "own_VLI_3_br_count",
    "own_VLI_4_br_count",
    "own_LI_studio_count",
    "own_LI_1_br_count",
    "own_LI_2_br_count",
    "own_LI_3_br_count",
    "own_LI_4_br_count",
    "own_MI_studio_count",
    "own_MI_1_br_count",
    "own_MI_2_br_count",
    "own_MI_3_br_count",
    "own_MI_4_br_count",
    "own_MoI_studio_count",
    "own_MoI_1_br_count",
    "own_MoI_2_br_count",
    "own_MoI_3_br_count",
    "own_MoI_4_br_count",
    "own_HI_studio_count",
    "own_HI_1_br_count",
    "own_HI_2_br_count",
    "own_HI_3_br_count",
    "own_HI_4_br_count"]].copy()
puma_counts_df

,rent_ELI_studio_count,rent_ELI_1_br_count,rent_ELI_2_br_count,rent_ELI_3_br_count,rent_ELI_4_br_count,rent_VLI_studio_count,rent_VLI_1_br_count,rent_VLI_2_br_count,rent_VLI_3_br_count,rent_VLI_4_br_count,rent_LI_studio_count,rent_LI_1_br_count,rent_LI_2_br_count,rent_LI_3_br_count,rent_LI_4_br_count,rent_MI_studio_count,rent_MI_1_br_count,rent_MI_2_br_count,rent_MI_3_br_count,rent_MI_4_br_count,rent_MoI_studio_count,rent_MoI_1_br_count,rent_MoI_2_br_count,rent_MoI_3_br_count,rent_MoI_4_br_count,rent_HI_studio_count,rent_HI_1_br_count,rent_HI_2_br_count,rent_HI_3_br_count,rent_HI_4_br_count,own_ELI_studio_count,own_ELI_1_br_count,own_ELI_2_br_count,own_ELI_3_br_count,own_ELI_4_br_count,own_VLI_studio_count,own_VLI_1_br_count,own_VLI_2_br_count,own_VLI_3_br_count,own_VLI_4_br_count,own_LI_studio_count,own_LI_1_br_count,own_LI_2_br_count,own_LI_3_br_count,own_LI_4_br_count,own_MI_studio_count,own_MI_1_br_count,own_MI_2_br_count,own_MI_3_br_count,own_MI_4_br_count,own_MoI_studio_count,own_MoI_1_br_count,own_MoI_2_br_count,own_MoI_3_br_count,own_MoI_4_br_count,own_HI_studio_count,own_HI_1_br_count,own_HI_2_br_count,own_HI_3_br_count,own_HI_4_br_count
PUMA,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
101,205.00,1604.00,1840.00,138.00,188.00,1447.00,3319.00,3353.00,605.00,391.00,2334.00,6327.00,7361.00,2389.00,531.00,2962.00,8630.00,9111.00,2495.00,657.00,3215.00,10219.00,9987.00,2835.00,805.00,173.00,771.00,979.00,473.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,61.00,0.00,0.00,321.00,33.00,126.00,0.00,0.00,376.00,33.00,350.00,0.00,0.00,573.00,596.00,536.00,0.00,111.00,223.00,530.00,518.00
102,2790.00,4483.00,1963.00,1668.00,73.00,5236.00,9342.00,6660.00,2635.00,603.00,6630.00,14605.00,12561.00,4423.00,710.00,7437.00,17735.00,14144.00,5357.00,710.00,7517.00,21378.00,16005.00,5603.00,1163.00,1061.00,3189.00,900.00,183.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,72.00,0.00,0.00,0.00,142.00,376.00,237.00,0.00,0.00,260.00,424.00,315.00,0.00,0.00,260.00,424.00,315.00,0.00,130.00,372.00,58.00,62.00
103,541.00,798.00,212.00,241.00,193.00,937.00,1822.00,1598.00,490.00,554.00,1959.00,5326.00,4344.00,860.00,972.00,2617.00,5908.00,5562.00,1260.00,1695.00,2808.00,6784.00,5980.00,1775.00,1695.00,109.00,400.00,365.00,178.00,0.00,0.00,0.00,59.00,0.00,74.00,0.00,0.00,59.00,65.00,206.00,0.00,0.00,162.00,378.00,396.00,0.00,0.00,162.00,461.00,582.00,0.00,0.00,223.00,729.00,930.00,0.00,0.00,716.00,1114.00,1113.00
104,133.00,1217.00,2334.00,1424.00,0.00,783.00,3955.00,6744.00,2348.00,473.00,1332.00,6194.00,10541.00,4536.00,1002.00,1472.00,6194.00,10950.00,4654.00,1097.00,1472.00,6194.00,11054.00,4654.00,1097.00,62.00,0.00,182.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,679.00,0.00,0.00,0.00,59.00,941.00,0.00,0.00,0.00,252.00,941.00,0.00,0.00,0.00,300.00,941.00,63.00,0.00,141.00,174.00,280.00,0.00
105,278.00,800.00,424.00,397.00,226.00,482.00,2612.00,2473.00,1760.00,602.00,1014.00,6553.00,8467.00,3459.00,812.00,1116.00,7776.00,10583.00,4270.00,812.00,1297.00,8803.00,12121.00,4654.00,878.00,573.00,908.00,468.00,101.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,59.00,70.00,0.00,0.00,0.00,233.00,613.00,377.00,0.00,0.00,379.00,718.00,377.00,0.00,0.00,761.00,1199.00,596.00,0.00,0.00,331.00,633.00,235.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11103,69.00,864.00,1304.00,174.00,114.00,279.00,1791.00,2773.00,880.00,310.00,1030.00,5239.00,8278.00,4808.00,2250.00,1128.00,5966.00,9273.00,5374.00,2366.00,1128.00,6107.00,9558.00,5478.00,2366.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,69.00,0.00,0.00,0.00,0.00,69.00,0.00,0.00,274.00,318.00,148.00,0.00,0.00,274.00,394.00,252.00,0.00,0.00,274.00,607.00,252.00,0.00,0.00,61.00,0.00,0.00
11104,934.00,488.00,158.00,190.00,0.00,934.00,852.00,1123.00,486.00,252.00,1367.00,5680.00,7799.00

In [45]:
#export it
puma_counts_df.to_csv("weighted_puma_counts.csv")

In [46]:
# for recent units
recent_puma_sums_df = recent_units_df.groupby("PUMA").sum()
recent_puma_counts_df = recent_puma_sums_df[["rent_ELI_studio_count","rent_ELI_1_br_count",
    "rent_ELI_2_br_count",
    "rent_ELI_3_br_count",
    "rent_ELI_4_br_count",
    "rent_VLI_studio_count",
    "rent_VLI_1_br_count",
    "rent_VLI_2_br_count",
    "rent_VLI_3_br_count",
    "rent_VLI_4_br_count",
    "rent_LI_studio_count",
    "rent_LI_1_br_count",
    "rent_LI_2_br_count",
    "rent_LI_3_br_count",
    "rent_LI_4_br_count",
    "rent_MI_studio_count",
    "rent_MI_1_br_count",
    "rent_MI_2_br_count",
    "rent_MI_3_br_count",
    "rent_MI_4_br_count",
    "rent_MoI_studio_count",
    "rent_MoI_1_br_count",
    "rent_MoI_2_br_count",
    "rent_MoI_3_br_count",
    "rent_MoI_4_br_count",
    "rent_HI_studio_count",
    "rent_HI_1_br_count",
    "rent_HI_2_br_count",
    "rent_HI_3_br_count",
    "rent_HI_4_br_count",
    "own_ELI_studio_count",
    "own_ELI_1_br_count",
    "own_ELI_2_br_count",
    "own_ELI_3_br_count",
    "own_ELI_4_br_count",
    "own_VLI_studio_count",
    "own_VLI_1_br_count",
    "own_VLI_2_br_count",
    "own_VLI_3_br_count",
    "own_VLI_4_br_count",
    "own_LI_studio_count",
    "own_LI_1_br_count",
    "own_LI_2_br_count",
    "own_LI_3_br_count",
    "own_LI_4_br_count",
    "own_MI_studio_count",
    "own_MI_1_br_count",
    "own_MI_2_br_count",
    "own_MI_3_br_count",
    "own_MI_4_br_count",
    "own_MoI_studio_count",
    "own_MoI_1_br_count",
    "own_MoI_2_br_count",
    "own_MoI_3_br_count",
    "own_MoI_4_br_count",
    "own_HI_studio_count",
    "own_HI_1_br_count",
    "own_HI_2_br_count",
    "own_HI_3_br_count",
    "own_HI_4_br_count"]].copy()
recent_puma_counts_df

,rent_ELI_studio_count,rent_ELI_1_br_count,rent_ELI_2_br_count,rent_ELI_3_br_count,rent_ELI_4_br_count,rent_VLI_studio_count,rent_VLI_1_br_count,rent_VLI_2_br_count,rent_VLI_3_br_count,rent_VLI_4_br_count,rent_LI_studio_count,rent_LI_1_br_count,rent_LI_2_br_count,rent_LI_3_br_count,rent_LI_4_br_count,rent_MI_studio_count,rent_MI_1_br_count,rent_MI_2_br_count,rent_MI_3_br_count,rent_MI_4_br_count,rent_MoI_studio_count,rent_MoI_1_br_count,rent_MoI_2_br_count,rent_MoI_3_br_count,rent_MoI_4_br_count,rent_HI_studio_count,rent_HI_1_br_count,rent_HI_2_br_count,rent_HI_3_br_count,rent_HI_4_br_count,own_ELI_studio_count,own_ELI_1_br_count,own_ELI_2_br_count,own_ELI_3_br_count,own_ELI_4_br_count,own_VLI_studio_count,own_VLI_1_br_count,own_VLI_2_br_count,own_VLI_3_br_count,own_VLI_4_br_count,own_LI_studio_count,own_LI_1_br_count,own_LI_2_br_count,own_LI_3_br_count,own_LI_4_br_count,own_MI_studio_count,own_MI_1_br_count,own_MI_2_br_count,own_MI_3_br_count,own_MI_4_br_count,own_MoI_studio_count,own_MoI_1_br_count,own_MoI_2_br_count,own_MoI_3_br_count,own_MoI_4_br_count,own_HI_studio_count,own_HI_1_br_count,own_HI_2_br_count,own_HI_3_br_count,own_HI_4_br_count
PUMA,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
101,50.00,40.00,145.00,0.00,0.00,256.00,305.00,437.00,74.00,64.00,462.00,1428.00,1411.00,1120.00,151.00,794.00,2030.00,2237.00,1159.00,277.00,1009.00,3105.00,2656.00,1333.00,425.00,91.00,321.00,656.00,122.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
102,314.00,319.00,0.00,575.00,73.00,672.00,390.00,473.00,684.00,73.00,1142.00,1741.00,1739.00,819.00,125.00,1368.00,3486.00,2250.00,1211.00,125.00,1448.00,5351.00,3160.00,1302.00,125.00,428.00,1033.00,450.00,183.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
103,0.00,439.00,0.00,0.00,0.00,0.00,439.00,0.00,76.00,0.00,102.00,977.00,776.00,350.00,95.00,564.00,1105.00,1363.00,691.00,699.00,662.00,1772.00,1481.00,1011.00,699.00,109.00,288.00,260.00,137.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
104,0.00,0.00,104.00,139.00,0.00,0.00,204.00,517.00,217.00,135.00,109.00,858.00,1131.00,952.00,356.00,109.00,858.00,1288.00,1009.00,451.00,109.00,858.00,1329.00,1009.00,451.00,0.00,0.00,182.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,26.00,127.00,0.00
105,185.00,198.00,83.00,0.00,0.00,185.00,501.00,83.00,180.00,113.00,185.00,1093.00,1468.00,282.00,113.00,185.00,1303.00,2526.00,463.00,113.00,366.00,1469.00,2998.00,604.00,179.00,0.00,597.00,301.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,64.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11103,69.00,0.00,29.00,0.00,0.00,69.00,48.00,224.00,0.00,0.00,190.00,1072.00,1179.00,713.00,123.00,190.00,1371.00,1660.00,1123.00,123.00,190.00,1426.00,1660.00,1123.00,123.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
11104,55.00,0.00,0.00,0.00,0.00,55.00,99.00,0.00,0.00,0.00,243.00,1601.00,805.00,140.00,311.00,243.00,1706.00,1101.00,754.00,311.00,243.00,1909.00,1148.00,853.00,311.00,0.00,0.00,0.00,124.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,50.00,0.00
11105,0.00,0.00,274.00,0.00,0.

In [47]:
#export it
recent_puma_counts_df.to_csv("weighted_recent_puma_counts.csv")